<a href="https://colab.research.google.com/github/Datamin-D/Projects/blob/main/%EC%97%90%EB%B8%8C%EB%A6%AC%ED%83%80%EC%9E%84_%EA%B0%95%EC%9D%98%ED%8F%89%EA%B0%80_%ED%81%AC%EB%A1%A4%EB%9F%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 에브리 타임 강의평가 크롤러

In [ ]:
import requests
from urllib.request import urlopen, urlretrieve
from bs4 import BeautifulSoup
import time
from random import randint
import re
import pandas as pd
import pickle
from tqdm import tqdm
import os

from random import *
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common import exceptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import numpy as np


# driver = webdriver.Chrome(r"C:\Users\MinSeong\Desktop\chrome driver\chromedriver.exe")
# wait = WebDriverWait(driver, 5)
# driver.get('https://everytime.kr/login')

# 1. 데이터 수집

In [ ]:
#1. 로그인 페이지

driver = webdriver.Chrome(r"C:\Users\MinSeong\Desktop\chrome driver\chromedriver.exe")
wait = WebDriverWait(driver, 5)
driver.get('https://everytime.kr/login')

In [ ]:
#2. 로그인

etid = '아이디를 여기 입력해주세요'
etpw = '패스워드를 여기 입력해주세요'

elem = driver.find_element_by_css_selector("#container > form > p:nth-child(1) > input")
elem.send_keys(etid)

elem = driver.find_element_by_css_selector("#container > form > p:nth-child(2) > input")
elem.send_keys(etpw)

elem.send_keys(Keys.RETURN) #엔터키
time.sleep(0.5)

In [ ]:
#3. 최근 20 강의평가
lecurl = 'https://everytime.kr/lecture'
driver.get(lecurl)

In [ ]:
#4. 강의평가 url 수집
#스크롤 내리는 코드
# 로그인 후 강의평가 게시판 클릭 완료한 상태에서 실행.
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
for i in range(15):
    ActionChains(driver).key_down(Keys.END).perform() # 스크롤 내리는 코드
    time.sleep(random.random() + 0.2)

In [ ]:
#5. 강의 평가 url 수집

# container
container = driver.find_elements_by_css_selector('a.article')
#container
# 실험용
#container = container[:3]
urls = [] #최근강의200개 url을 담을 list
for cont in container:
    urls.append(cont.get_attribute('href')) #보시면 href에 해당 게시글의 link가 있습니다.
urls # 정상수집 확인

In [ ]:
'#container > div.side.head > p:nth-child(3) > span'

,lecture,star,content


In [ ]:
#6. 강의 평가 수집 - 과목 이름, 교수명, 

df = pd.DataFrame(data = [], columns = ['lecture','prof','star','contents'])
for i in range(len(urls)-197):
    driver.get(urls[i]) # 과목페이지 접근
    time.sleep(random.random() + 0.5) 
    
    lecture = np.nan
    lecture = driver.find_element_by_css_selector('#container > div.side.head > h2').text
    
    prof = np.nan
    prof = driver.find_element_by_css_selector('#container > div.side.head > p:nth-child(3) > span').text
    #print(lecture, prof) 정상 작동
    # 강의 평가들
    rvs = driver.find_elements_by_css_selector('#container > div.side.article > div.articles > article')
    empty_df = pd.DataFrame(data = [], columns = ['lecture','prof','star','contents'])
    for rv_no,rv in enumerate(rvs):
        star = np.nan
        star = rv.find_element_by_css_selector('p.rate > span > span').get_attribute('style')
        
        content = np.nan
        content = rv.find_element_by_css_selector('p.text').text
        empty_df.loc[rv_no] = lecture,prof,star,content # 작동 확인
        df = df.append(empty_df)
        df.reset_index(drop = True, inplace = True)

# 2. 전처리

In [ ]:
# 7. 전처리
df = df.drop_duplicates()
df.reset_index(drop = True, inplace = True)

In [ ]:
#별점 전처리
df['star'].unique()

array(['width: 100%;', 'width: 40%;', 'width: 60%;', 'width: 80%;',
       'width: 20%;'], dtype=object)

In [ ]:
def extractor(x):
    p = re.compile('([0-9])*\%')
    s = p.search(x)
    star = s.group()
    return star

df['star'] = df['star'].apply(lambda x: extractor(x))
df['star'].unique()

In [ ]:

def star_changer(x):
    if x == '100%':
        star = 5
    elif x == '80%':
        star = 4
    elif x == '60%':
        star = 3
    elif x == '40%':
        star = 2
    elif x == '20%':
        star = 1
    return star
df['star'] = df['star'].apply(lambda x: star_changer(x))

In [ ]:
df['star'].value_counts() # star 분포확인
#df

5    363
4     70
3     21
2      4
1      2
Name: star, dtype: int64